In [2]:
import pandas
import math

data = pandas.read_csv('write.csv')
dataa = data[data.actionCategory =='Tag filtering Clicked']


In [3]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [4]:
def getData(data, col1, col2):
    xAxis = []
    yAxis = []
    fields = []
    g = data.groupby([col2, col1]).size()
    g1=g.unstack()
    g1=g1.fillna(0.0)
    for x in g1:
        for y_name, y in g1[x].iteritems():
            yAxis.append(x)
            fields.append(y)
            xAxis.append(y_name)
    obj = {"xAxis":xAxis, "yAxis":yAxis, "fields":fields}
    return obj

In [5]:
from bokeh.layouts import row
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.models import HoverTool
from bokeh.plotting import ColumnDataSource
import bokeh
import math
output_notebook() 

Loading BokehJS ...

In [6]:
def dbgroupbyplot(data, grb1, grb2, title='', height = 1000, width = 1000):
    obj = getData(data,grb1,grb2)
    fields = obj["fields"]
    xAxis = obj["xAxis"]
    yAxis = obj["yAxis"]

    colors = []
    maximum = max(fields)
    for c in fields:
        colors.append(bokeh.colors.RGB(255,(1-math.sqrt(c/maximum))*255,255))

    desc = list(map(lambda x:str(x),fields))
    source = ColumnDataSource(data=dict(
        y = yAxis, 
        x= xAxis,
        color=colors,
        desc = desc
    ))
    hover = HoverTool(tooltips=[
        (grb2, "@y"),
        ("tool", "@x"),
        ("numbers", "@desc")
    ])
    hm = figure(title=title, tools=[hover,  'wheel_zoom', 'box_zoom', 'pan', 'reset'],
                y_range=data[grb1].unique(), x_range=data[grb2].unique(),  plot_width=1000, plot_height=1000)

    hm.rect(     y = 'y', x = 'x', source = source, color="color", width=1, height=1)
    hm.xaxis.major_label_orientation = math.pi/3

    show(hm)  # open a browser

In [7]:
data = pandas.read_csv('write.csv')
category1 = data[data.actionCategory == 'Category downloaded'].groupby('actionName').size()
category2=data[data.actionCategory == 'Category filtering Clicked'].groupby('actionName').size()
category3=data[data.actionCategory == 'CategoryChip filtering Clicked'].groupby('actionName').size()
allCategoryFiltering = category2 + category3 + category1.values

tagsCount = data[data.actionCategory=='Tag filtering Clicked'].groupby('actionName').size().sort_values()

departments = data[data.actionCategory == 'WebNei clicked'].groupby('department').size().sort_values()

In [8]:
import pandas
import numpy as np
import math
import functools
import bokeh
from bokeh.core.properties import value
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import output_notebook
from collections import Counter
output_notebook()








def plot(series, title="",y_axis_label="ilość", width=900, height=750):
    p = figure(title=title, x_axis_label=series.index.name, y_axis_label='ilość', x_range=series.index.values, plot_width=width, plot_height=height)
    p.xaxis.major_label_orientation = math.pi/5
    p.vbar(series.index.values, line_width=2,top=series.values,width=0.9)
    p.xaxis.major_label_text_font_size = "9pt"
    show(p)

Loading BokehJS ...

In [9]:
plot(allCategoryFiltering.sort_values(), 'Category filtering', )

In [10]:
dbgroupbyplot(data[data.actionCategory == 'Category downloaded'], 'actionName', 'department', 'Category count in departments')

In [11]:
plot(tagsCount, 'Tags filtering')

In [12]:
dbgroupbyplot(dataa, 'actionName', 'department', 'Tag count in departments')

In [13]:
print('Top 3 departments have ' + str(departments.tail(3).sum()/departments.sum() * 100) + '% of all departments')

Top 3 departments have 94.6924529502875% of all departments


In [14]:
plot(departments.tail(6), 'Top 6 departments')

In [15]:
plot(departments.head(departments.size - 6), 'Rest departments')

In [16]:
dep = pandas.DataFrame(data.drop_duplicates('idVisit').groupby(['department', 'country']).size(), columns = ['count'])
g = dep['count'].groupby(level=0, group_keys=False)
res = g.apply(lambda x: x.sort_values(ascending=False).head(3))

In [17]:
res

department                                  country       
AA Applications &amp; Analytics             India              120
                                            Finland            103
                                            USA                 88
BLCTO Bell Labs &amp; CTO                   Germany              5
                                            USA                  4
                                            Belgium              2
CFO Chief Financial Office                  India                1
CIOO Chief Innovation&amp;Operating Office  Finland             15
                                            Germany              7
                                            India                6
CO Customer Operations                      China             4343
                                            India              711
                                            Taiwan             376
COO Chief Operating Office                  Italy                6
   

In [18]:
plot(data.drop_duplicates('idVisit').groupby('country').size().sort_values().tail(10), 'Countries by unique visit')

In [19]:
plot(data[data.actionCategory == 'WebNei clicked'].groupby('country').size().sort_values().tail(10), 'Countries by \'WebNei Clicked\' action')

In [20]:
plot(data.groupby('country').size().sort_values().tail(10), 'Countries by all actions')

In [21]:
plot(data.drop_duplicates('userName').groupby('country').size().sort_values().tail(10), 'Countries by unique users')